<a href="https://colab.research.google.com/github/giuseppe-iacono-eng/telco-churn-prediction/blob/main/02_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>